## Example of reading Apex Pro Client execution and allocation files

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Load executions 
etdx_trades_df = pd.read_csv('/Users/avanthavasi/Downloads/ETDX_20230112_1_TRADE.csv')
etdx_trades_df.shape

In [ ]:
etdx_trades_df.info()

In [ ]:
etdx_trades_df.head()

In [ ]:
etdx_trades_df.type.value_counts()

In [ ]:
# Data clean-up
# This is needed for an exercise in cells below. It is cleaning up null values, which pandas will generally populate as NaN, which is not very useful
etdx_trades_df.loc[:, 'side.qualifier'].fillna('', inplace=True)
etdx_trades_df.loc[:, 'contra_side_qualifier'].fillna('', inplace=True)

In [ ]:
# split trades between allocations and executions
cond = etdx_trades_df['type'] == 'allocation_trade'
alloc_df = etdx_trades_df.loc[cond]
exec_df = etdx_trades_df.loc[~cond]

In [ ]:
# Number of executions by ticker
exec_df['instrument.identifier'].value_counts()

In [ ]:
# Number of allocations by ticker
alloc_df['instrument.identifier'].value_counts()

In [ ]:
# Let's look at one symbol - "A"
# Allocations
alloc_df.loc[alloc_df['instrument.identifier'] == 'A']

In [ ]:
# Executions 
exec_df.loc[exec_df['instrument.identifier'] == 'A']

In [ ]:
# Verify that the allocations they sent for trades in 'A' reconcile with the individual executions
# Trade Workflow
# 1. Execution Trade: Ledger entry 1 for house account + Ledger Entry 2 for Avg price account (direction in avg price account = direction of trade)
# 2. Allocation trade: Ledger entry 1 removes trade from avg price account (direction in avg price account is opp of direction of trade) + Ledger Entry 2 moves trade into customer account

exec_a_buys_df = exec_df.loc[(exec_df['instrument.identifier'] == 'A') & (exec_df['side.direction'] == 'buy')]

In [ ]:
exec_a_buys_df['quantity'].sum()

In [ ]:
# Weighted average price of executions
def weighted_avg_price(df):    
    return np.average(df['price'], weights=df['quantity'])

In [ ]:
weighted_avg_price(exec_a_buys_df)

In [ ]:
# Suppose we wanted to do this across all names / directions and qualfiers.
# we can apply grouping to get weighted avg prices across all executions by [ticker, direction, contra_side_qualifier]
weighted_exec_prices = exec_df.groupby(['instrument.identifier', 'side.direction', 'side.qualifier'])[['quantity', 'price']].apply(weighted_avg_price).reset_index()
weighted_exec_prices.columns = ['instrument.identifier', 'side.direction', 'side.qualifier', 'price']

In [ ]:
weighted_exec_prices

In [ ]:
alloc_prices = alloc_df.loc[:, ['instrument.identifier', 'side.direction', 'contra_side_qualifier', 'price']]
alloc_prices.columns = ['instrument.identifier', 'side.direction', 'side.qualifier', 'price']

In [ ]:
# flip directions
alloc_prices.loc[:, 'side.direction'] = alloc_prices.loc[:, 'side.direction'].apply(lambda dir: 'buy' if dir == 'sell' else 'sell')

In [ ]:
alloc_prices

In [ ]:
merged = weighted_exec_prices.merge(alloc_prices, how='inner', on=['instrument.identifier', 'side.direction', 'side.qualifier'], suffixes=['_exec', '_alloc'])

In [ ]:
merged['diff'] = abs(merged['price_exec'] - merged['price_alloc'])

In [ ]:
merged.sort_values('diff', ascending=False)

In [ ]:
merged.loc[merged['diff'] > 0.05]